In [1]:
import os
import cv2
import numpy as np
import os
from random import shuffle
from keras.layers import Conv2D, Dense, Flatten,MaxPooling2D,Dropout
from keras.models import Sequential, load_model
from keras.optimizers import Adam

Using Theano backend.
WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Users\SOUMYA\Anaconda3\envs\summer_project\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
C:\Users\SOUMYA\Anaconda3\envs\summer_project\lib\site-packages\scipy\__init__.py:115: UserWarning: Numpy 1.13.3 or above is required for this version of scipy (detected version 1.11.3)
  UserWarning)
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
TRAIN_AUTH_DIR = r'dataset\train\authentic'
TRAIN_FORG_DIR = r'dataset\train\forged'

TEST_DIR = r'dataset\test'
IMG_ROW, IMG_COL = 159, 118

In [3]:
def create_train_data():
    training_data = []
    for img in os.listdir(TRAIN_AUTH_DIR):
        if img[0] != '.' : 
            path = os.path.join(TRAIN_AUTH_DIR, img)
            img_data = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            img_data = cv2.divide(img_data, 255)
            img_data = cv2.resize(img_data, (IMG_ROW, IMG_COL))
            training_data.append([np.array(img_data), 1])
    for img in os.listdir(TRAIN_FORG_DIR):
        if img[0] != '.' : 
            path = os.path.join(TRAIN_FORG_DIR, img)
            img_data = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            img_data = cv2.divide(img_data, 255)
            img_data = cv2.resize(img_data, (IMG_ROW, IMG_COL))
            training_data.append([np.array(img_data), 0])
    
    shuffle(training_data)
    np.save('metadata/train_data.npy', training_data)
    return training_data

imgname = []
def create_test_data():
    testing_data = []
    for img in os.listdir(TEST_DIR):
        if img[0] != '.' :
            path = os.path.join(TEST_DIR, img)
            imgname.append(img)
            img_data = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            img_data = cv2.divide(img_data, 255)
            img_data = cv2.resize(img_data, (IMG_ROW, IMG_COL))
            testing_data.append(np.array(img_data))

    #shuffle(testing_data)
    np.save('metadata/test_data.npy', testing_data)
    return testing_data


train_file = r'metadata/train_data.npy'
test_file = r'metadata/test_data.npy'

if os.path.exists(train_file) and os.path.exists(test_file):
    train_data = np.load(train_file, allow_pickle=True)
    test_data = np.load(test_file, allow_pickle=True)
    for img in os.listdir(TEST_DIR):
        if img[0] != '.' :
            imgname.append(img)
    print('Found numpy data on drive. Loaded.')
    
else:
    print('Creating numpy data.')
    train_data = create_train_data()
    test_data = create_test_data()
    print('Numpy data created.')
    
print('You may proceed!')

Found numpy data on drive. Loaded.
You may proceed!


In [4]:
X_train = np.array([x[0] for x in train_data]).reshape(-1, IMG_ROW, IMG_COL, 1)
y_train = np.array([x[1] for x in train_data])

X_test = np.array([x for x in test_data]).reshape(-1, IMG_ROW, IMG_COL, 1)

if os.path.exists('metadata/model_1_2.h5'):
    model = load_model('metadata/model_1_2.h5')
    print('Found saved model on drive. Loaded.')

else:
    print('Building model.')    
    model = Sequential()

    model.add(Conv2D(16, input_shape=(IMG_ROW, IMG_COL, 1), kernel_size=(5,5), activation='relu'))
    model.add(Conv2D(32, kernel_size=(5,5), activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(64, kernel_size=(5,5), activation='relu'))
    
    model.add(Flatten())
   # model.add(Dense(100, activation='tanh'))
    
    model.add(Dense(50, activation='tanh'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    
    print('Compiling model.')
    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    
print('You may proceed!')

C:\Users\SOUMYA\Anaconda3\envs\summer_project\lib\site-packages\theano\tensor\nnet\conv.py:98: UserWarning: theano.tensor.nnet.conv.conv2d is deprecated. Use theano.tensor.nnet.conv2d instead.
  warnings.warn("theano.tensor.nnet.conv.conv2d is deprecated."


Found saved model on drive. Loaded.
You may proceed!


In [ ]:
# Fitting the model

print('Fitting Model. Please Wait.')
model.fit(X_train, y_train, batch_size=2, epochs=5, verbose=1)

print('Model Fitting Finished.')

model.save('metadata/model_1_2.h5')
print('Model Saved.')
print('You may proceed!')

Fitting Model. Please Wait.
Epoch 1/5


In [18]:
output = model.predict(X_test)

print('Authenticity predicted : ')
for i in range(len(output)):
    print(imgname[i], ": %.3f %%" % (output[i]*100))

Authenticity predicted : 
img 2019-08-18 14-51-16.png : 99.977 %
img 2019-10-02 13-19-34.png : 99.977 %
img 2019-10-14 12-41-06.png : 0.132 %
img 2019-10-14 12-50-46.png : 93.336 %
img 2019-10-14 15-29-59.png : 28.555 %
img 2019-10-15 18-41-59.png : 99.977 %
img 2019-10-15 19-01-33.png : 99.978 %
img 2019-10-28 17-27-02.png : 0.022 %
img 2019-10-28 17-39-32.png : 0.025 %
img 2019-10-28 20-33-37.png : 0.024 %
img 2019-10-28 20-35-54.png : 0.022 %
